# Logic in Sole.jl

In [ ]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
Pkg.update()

## SoleLogics.jl

`SoleLogics.jl` is not only the package in the `Sole.jl` framework specifically
developed for logic: it is the core library of the framework itself.

In a nutshell, it provides a fresh codebase for computational logic, featuring
easy manipulation of:
- Propositional and (multi)modal logics (atoms, logical constants, alphabets,
grammars, crisp/fuzzy algebras);
- Logical formulas (parsing, random generation, minimization);
- Logical interpretations (propositional valuations, Kripke structures);
- Algorithms for finite
[model checking](https://en.wikipedia.org/wiki/Model_checking), that is,
checking that a formula is satisfied by an interpretation.

In this notebook, we will see examples of all these functionalities, providing
a comprehensive overview of the whole package.

In [ ]:
using SoleLogics

### Propositional Logic

Parsing and manipulating Formulas.

In [ ]:
φ1 = parseformula("¬p∧q∨(s∨z)")

In [ ]:
syntaxstring(φ1; parenthesize_commutatives = true)

In [ ]:
filter(ψ -> height(ψ) == 1, subformulas(φ1))

In [ ]:
filter(ψ -> natoms(ψ) == 1, subformulas(φ1))

In [ ]:
φ2 = ⊥ ∨ Atom("t") → φ1

Generating random formulas.

In [ ]:
alphabet = @atoms p q

In [ ]:
SoleLogics.BASE_PROPOSITIONAL_CONNECTIVES

In [ ]:
using Random

h = 2;   # the height of the formula

randformula(
    Random.MersenneTwister(507),    # the random number generator we want to use
    h,
    alphabet,
    SoleLogics.BASE_PROPOSITIONAL_CONNECTIVES
)

Model checking.

In [ ]:
φ1 = parseformula("¬(p ∧ q)")

In [ ]:
I = TruthDict(["p" => true, "q" => false])

In [ ]:
check(φ1, I)

In [ ]:
φ2 = parseformula("¬(p ∧ q) ∧ (r ∨ q)")

In [ ]:
interpret(φ2, I)

### Modal Logic K

Generating random formulas.

In [ ]:
SoleLogics.BASE_MODAL_CONNECTIVES

In [ ]:
randformula(
    Random.MersenneTwister(4267),
    h,
    alphabet,
    SoleLogics.BASE_MODAL_CONNECTIVES
)

Model checking.

In [ ]:
using Graphs

# Instantiate a Kripke frame with 5 worlds and 5 edges
worlds = World.(1:5)
edges = Edge.([(1,2), (1,3), (2,4), (3,4), (3,5)])
fr = SimpleModalFrame(worlds, Graphs.SimpleDiGraph(edges))

In [ ]:
# Enumerate the worlds that are accessible from the first world
accessibles(fr, first(worlds))

In [ ]:
# Assign each world a propositional interpretation
@atoms p q
valuation = Dict([
    worlds[1] => TruthDict([p => true, q => false]),
    worlds[2] => TruthDict([p => true, q => true]),
    worlds[3] => TruthDict([p => true, q => false]),
    worlds[4] => TruthDict([p => false, q => false]),
    worlds[5] => TruthDict([p => false, q => true]),
])

# Instantiate a Kripke structure by combining a Kripke frame and the
# propositional interpretations over each world
K = KripkeStructure(fr, valuation)

In [ ]:
# Generate a modal formula
φ = parseformula("◊(p ∧ q)");

# Check the just generated formula on each world of the Kripke structure
[w => check(φ, K, w) for w in worlds]

### Temporal Modal Logics

Linear Temporal Logic (LTL).

In [ ]:
# A frame consisting of 10 (evenly spaced) points
fr = FullDimensionalFrame((10,), Point{1, Int64})
allworlds(fr) |> collect

In [ ]:
# Linear Temporal Logic (LTL) `successor` relation
accessibles(fr, Point(3), SoleLogics.SuccessorRel) |> collect

In [ ]:
# Linear Temporal Logic (LTL) `greater than` (i.e., future) relation
accessibles(fr, Point(3), SoleLogics.GreaterRel) |> collect

Halpern and Shoham's Interval Temporal Logic (HS).

In [ ]:
# An interval frame consisting of all intervals over 10 (evenly spaced) points
fr = FullDimensionalFrame((10, ), Interval{Int64})
allworlds(fr) |> collect

In [ ]:
# Interval Algebra (IA) relation `L` (later)
accessibles(fr, Interval(3, 5), IA_L) |> collect

### Many-Valued Logic

## SoleReasoners.jl